In [46]:
from pyspark.sql import SparkSession

In [47]:
spark = (
    SparkSession.builder
    .appName("ETL_Job_Iceberg")
    .master('spark://spark-iceberg:7077')
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "rest")
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://rest:8181")
    .config("spark.sql.catalog.lakehouse_prod_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3://warehouse/wh/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.s3.endpoint", "http://minio:9000")
    .config("spark.eventLog.enabled", "true")
    .config("spark.eventLog.dir", "/home/iceberg/spark-events")
    .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")
    .config("spark.sql.defaultCatalog", "lakehouse_prod_catalog")
    .getOrCreate()
)


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCo

In [7]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-iceberg:7077 appName=ETL_Job_Iceberg>

In [25]:
spark.sql("CREATE DATABASE IF NOT EXISTS test_db")

DataFrame[]

In [26]:
spark.sql("DROP TABLE IF EXISTS test_db.test_table_001")

DataFrame[]

In [27]:
# Extract a Data
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.show()

[Stage 2:============================================>              (3 + 1) / 4]

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [29]:
df.write.mode("overwrite").format("parquet").saveAsTable("test_db.test_table_001")

In [34]:
spark.sql('DESCRIBE EXTENDED test_db.test_table_001').show()

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|            VendorID|       bigint|   NULL|
|tpep_pickup_datetime|timestamp_ntz|   NULL|
|tpep_dropoff_date...|timestamp_ntz|   NULL|
|     passenger_count|       double|   NULL|
|       trip_distance|       double|   NULL|
|          RatecodeID|       double|   NULL|
|  store_and_fwd_flag|       string|   NULL|
|        PULocationID|       bigint|   NULL|
|        DOLocationID|       bigint|   NULL|
|        payment_type|       bigint|   NULL|
|         fare_amount|       double|   NULL|
|               extra|       double|   NULL|
|             mta_tax|       double|   NULL|
|          tip_amount|       double|   NULL|
|        tolls_amount|       double|   NULL|
|improvement_surch...|       double|   NULL|
|        total_amount|       double|   NULL|
|congestion_surcharge|       double|   NULL|
|         airport_fee|       double|   NULL|
|         

In [37]:
spark.sql('SHOW TABLES').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [38]:
spark.sql('SELECT COUNT(*) as cnt FROM test_db.test_table_001').show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `test_db`.`test_table_001` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 28;
'Aggregate [count(1) AS cnt#651L]
+- 'UnresolvedRelation [test_db, test_table_001], [], false


In [40]:
%%sql

SELECT * FROM test_db.test_table_001

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2021-04-01 00:00:18,2021-04-01 00:21:54,1.0,8.4,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,2021-04-01 00:42:37,2021-04-01 00:46:23,1.0,0.9,1.0,N,75,236,2,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,0.0
1,2021-04-01 00:57:56,2021-04-01 01:08:22,1.0,3.4,1.0,N,236,168,2,11.5,3.0,0.5,0.0,0.0,0.3,15.3,2.5,0.0
1,2021-04-01 00:01:58,2021-04-01 00:54:27,1.0,0.0,1.0,N,47,61,1,44.2,0.0,0.5,0.0,0.0,0.3,45.0,0.0,0.0
2,2021-04-01 00:24:55,2021-04-01 00:34:33,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0
2,2021-04-01 00:19:16,2021-04-01 00:21:46,1.0,0.77,1.0,N,142,238,1,4.5,0.5,0.5,1.24,0.0,0.3,9.54,2.5,0.0
2,2021-04-01 00:25:11,2021-04-01 00:31:53,1.0,3.65,1.0,N,238,244,1,11.5,0.5,0.5,2.56,0.0,0.3,15.36,0.0,0.0
1,2021-04-01 00:27:53,2021-04-01 00:47:03,0.0,8.9,1.0,N,138,239,1,26.5,3.0,0.5,7.25,6.12,0.3,43.67,2.5,0.0
2,2021-04-01 00:24:24,2021-04-01 00:37:50,1.0,2.98,1.0,N,151,244,2,12.0,0.5,0.5,0.0,0.0,0.3,13.3,0.0,0.0
1,2021-04-01 00:19:18,2021-04-01 00:41:25,1.0,8.9,1.0,N,132,196,2,28.0,0.5,0.5,0.0,0.0,0.3,29.3,0.0,0.0


In [44]:
spark.stop()